# 源文件检查程序

In [1]:
# 成立程序
import os
import re
import shutil
from datetime import datetime, date, timedelta
import win32com.client
import time
import pandas as pd
import openpyxl
import docx
import PyPDF4
import sys
import matplotlib.pyplot as plt
from docx.enum.table import WD_TABLE_ALIGNMENT
from docx.shared import RGBColor, Pt, Cm
from docx.enum.table import WD_ALIGN_VERTICAL
from docx.oxml.ns import qn
from copy import copy
from openpyxl.styles import Border, Side, Alignment, Font

start = time.time()
# while True:
# password = input('请输入授权人：')
# if password == '司进龙':
#    break
# else:
#    print('授权人输入错误！')
#    sys.exit(0)

if '成立' not in os.listdir('../'):
    os.makedirs('../成立/')
found_dir_path = '../成立/'  # 成立文件夹
follow_dir_path = './模板/'  # 成立模板文件夹
product_element_path = follow_dir_path + 'xxx-XX募集投资者明细+要素表.xlsx'  # 要素表
source_dir_path = './源文件/'
found_date = date.today() + timedelta(days=0)  # 产品成立日期
df = pd.read_excel(product_element_path, sheet_name='要素表',dtype={2: str}).set_index('产品名称')  # 读取要素表并以‘产品名称’列为标签列
product_list = df['产品简称'].dropna().values.tolist()  # 从要素表提取产品名简称列表
df_issuer = pd.read_excel(product_element_path, sheet_name='发行人', dtype=str).set_index('发行人简称')  # 以‘发行人简称’列为标签
issuer_list = df_issuer.index.dropna().values.tolist()  # 从要素表提取发行人简称列表
period_unit_list = ['次', '期', '开']
bank_water_select = ['次', '期', '开', '募集投资者明细', '改————']  # 筛选流水文件需回避的关键字
investor_index_list = ['客户名称', '客户姓名', '投资人姓名', '姓名*', '姓名', '姓名*\n（常规/文本）', '付款方别（户）名']
amount_index_list = ['金额', '万', '元', '规模（万）', '到账金额（万）', '认购金额（万元）', '认购金额*\n（常规/数值）', '认购金额（万）',
                     '认购金额（万元）\n（打款金额）', '认购金额（元）\n（打款金额）','挂牌金额（元）', '认购金额：万', '认购本金']
ID_type_index_list = ['证件类型', '证件类型\n（常规/文本）']
ID_num_index_list = ['证件编号', '证件号码', '证件号\n（常规/文本）']
bank_index_list = ['开户银行全称\n(打款账号的开户行)', '开户行\n（常规/文本）', '订单总行名称', '总行名称']
bank_num_index_list = ['银行账号\n（打款账号）', '开户行号\n（常规/文本）', '订单银行账号', '开户银行', '账号\n（常规/文本）', '银行账号']
w2c = 3  # 投资者明细excel模板（wb2）修改插入的列数


def num_to_cn(num):
    _MAPPING = (u'零', u'一', u'二', u'三', u'四', u'五', u'六', u'七', u'八', u'九', u'十', u'十一', u'十二', u'十三',
                u'十四', u'十五', u'十六', u'十七', u'十八', u'十九')
    _P0 = (u'', u'十', u'百')
    _S4 = 1000
    assert (0 <= num < _S4)
    if num < 20:
        return _MAPPING[num]
    else:
        lst = []
        while num >= 10:
            lst.append(num % 10)
            num = num / 10
        lst.append(num)
        c = len(lst)  # 位数
        result = u''
        for idx, val in enumerate(lst):
            val = int(val)
            if val != 0:
                result += _P0[idx] + _MAPPING[val]
                if idx < c - 1 and lst[idx + 1] == 0:
                    result += u'零'
        return result[::-1]


def source_all_file():
    for root, ds, fs in os.walk(source_dir_path):
        for f in fs:
            if f.endswith('.xlsx') or f.endswith('.xls'):
                f_path = os.path.join(root, f)
                yield f, f_path
                
                
def product_num_judge(p_file):
    num0 = ''
    num_dict = {'首': 1, '一': 1, '二': 2, '三': 3, '四': 4, '五': 5, '六': 6, '七': 7, '八': 8, '九': 9}
    p_file = p_file.replace('公开', '').replace('次开', '开').replace('期开', '开')
    for w in period_unit_list:  # 该文件包含“次”或“期”或"开"字（期数单位）
        if w in p_file:
            num0 = p_file[p_file.find(w) - 3: p_file.find(w)]  # 提取期数单位前三个字符
    if re.findall(r'(\d+)', num0):  # 三个字符中包含数字 为真即列表不为空
        num = int(re.findall(r'(\d+)', num0)[0])  # 提取数字即为期数
    elif '十' in num0:  # 三字符中包含‘十’
        if num0[2] == '十' and num0[1] in num_dict:  # ‘十’在三字符中第三位，且第二位有汉字数字 如“*五十”
            num = num_dict[num0[1]] * 10
        elif num0[2] == '十' and num0[1] not in num_dict:  # ‘十’在三字符中第三位,且第二位无汉字数字 如“**十”
            num = 10
        elif num0[1] == '十' and num0[0] in num_dict:  # ‘十’在三字符中第二位，且第一位为汉字数字 如“二十一”
            num1 = num_dict[num0[0]]
            num2 = num_dict[num0[2]]
            num = num1 * 10 + num2
        else:  # ‘十’在三字符中第二位，且第一位不为汉字数字 如“*十二”
            num = 10 + num_dict[num0[2]]
    else:
        num = num_dict[num0[2]]  # 三字符中不包含‘十’，也不包含数字,但包含汉字数字 如“**二”
    if '开' in p_file.replace('公开', '') and '首次开放' not in p_file:
        num = num + 1
    return num


def product_name_judge(name):  # 提取产品名简称
    p_name = '无法识别'
    name_judge = '唯一'
    p_issuer_name = '未能识别'
    for pi in product_list:
        if pi in name:
            p_name = pi
            break
    if any(word in file for word in period_unit_list):  # 该文件包含“次”或“期”或‘开’字（期数单位）
        if product_list.count(p_name) > 1:
            name_judge = '不唯一'
            num1, num2 = 0, 0
            p_num = product_num_judge(name)  # 判断文件名中期数
            sta_product_list = []
            if '统计表.xlsx' in os.listdir(found_dir_path):
                sta_df = pd.read_excel(found_dir_path + '统计表.xlsx', usecols = 'A:N', dtype = str)  # statistics统计表
                sta_product_list = sta_df['产品名称'].values.tolist()

            product_full_name1 = df[df['产品简称'] == p_name].index[0]  # 第一种产品名称
            if product_full_name1 in sta_product_list:
                # 第一种产品名称对应最大期数
                num1 = sta_df[sta_df['产品名称'] == product_full_name1]['产品统编号'].map(lambda x: int(x[-3:])).max()
                if p_num - num1 == 1:  # 为该产品名称的下一期
                    p_issuer_name = product_full_name1[:-2]  # 该产品名称对应发行人简称

            product_full_name2 = df[df['产品简称'] == p_name].index[1]  # 第二种产品名称
            if product_full_name2 in sta_product_list:
                # 第一种产品名称对应最大期数
                num2 = sta_df[sta_df['产品名称'] == product_full_name2]['产品统编号'].map(lambda x: int(x[-3:])).max()
                if p_num - num2 == 1:  # 为该产品名称的下一期
                    p_issuer_name = product_full_name2[:-2]  # 该产品名称对应发行人简称
                
    return p_name, name_judge, p_issuer_name


def issuer_judge(name):  # 提取发行人简称
    i_name = '未能识别'
    for si in issuer_list:
        if si in name:
            i_name = si
            break
    return i_name


def bank_water_merge(b_file, b_p_name, b_p_num, bp_sub_path, bc):
    b_wb1 = openpyxl.load_workbook(bp_sub_path + b_file)  # openpyxl读取明细源文件
    b_ws1 = b_wb1.active
    b_pf1 = pd.read_excel(bp_sub_path + b_file, skiprows=0)  # pandas读取明细源文件
    b_pf1.dropna(inplace=True, subset=[b_pf1.columns[bc-1]])  # 根据客户姓名列剔除空值行
    b_investor_list = b_pf1.iloc[:, bc-1].map(lambda x: x.strip().replace(' ', '')).tolist()
    b_w_name = '改————' + b_p_name + str(b_p_num) + '流水.xlsx'
    if os.path.isfile(bp_sub_path + b_w_name):  # 判断合并流水文件名指向文件是否已存在于产品子文件夹
        bank_water = pd.DataFrame(bp_sub_path + b_w_name)
    else:
        bank_water = pd.DataFrame()
    bi = 0
    for f, f_path0 in source_all_file():  # 筛选该产品流水源文件
        if b_p_name in f and not any(word in f for word in bank_water_select):  # and len(''.join((re.findall(r'[\u4e00-\u9fa5]+', f)))) < 6:  # 是否为该产品流水源文件
            f, f_path = to_xlsx(f, f_path0.replace(f, '', 1))
            wc = arrange_title(f, f_path.replace(f, '', 1))
            b_pf = pd.read_excel(f_path, header=0)  # 读取流水源文件
            b_water_list = b_pf.iloc[:, wc-1].tolist()  # 根据流水源文件生成投资者流水名单
            if any(word in b_water_list for word in b_investor_list) and list(b_pf)[0] == '核心流水号':  # 流水源文件是否含有该产品该期投资者名称
                bank_water = pd.concat([bank_water, pd.read_excel(f_path, header=0,
                                                                  dtype={2: str, 7: str, 10: str})])  # 合并流水
                shutil.copy(f_path0, bp_sub_path)  # 流水源文件移至产品子文件夹
                bi += 1
            os.remove(f_path)
    if bi > 0:
        bank_water.sort_values(by=['付款方银行', '交易金额'], ascending=False, na_position='last', inplace=True)
        bank_water.drop_duplicates(['核心流水号'], keep='first', inplace=True)
        bank_water.to_excel(bp_sub_path + b_w_name, index=False)  # 保存合并后的流水
    else:
        b_w_name = '无流水'

    return b_w_name


def clean_rows(sheet, row_num1, row_num2):
    for di in range(row_num1, row_num2):
        for dj in range(0, 31 + w2c):
            sheet[di][dj].value = None


def refresh(filename):
    xl_app = win32com.client.Dispatch("Excel.Application")
    xl_app.visible = False  # 是否显示excel窗口
    jwb = xl_app.Workbooks.Open(os.path.abspath(filename))
    jwb.RefreshAll()
    time.sleep(3)
    jwb.Save()
    jwb.Close()
    xl_app.Quit()


def search_bank(s_ws1, s_i, w_b_name):  # 搜索投资者明细源文件中的银行信息
    s_b_name = '空'
    s_b_num = '空'
    bank_list = ['分行', '营业部', '支行']
    nobank1_list = ['分行', '营业部', '支行', '银行卡支付']
    nobank2_list = ['无分行', '支行']
    if w_b_name is None:
        w_b_name = '空'
    w_b_name = w_b_name.replace('有限责任公司', '').replace('股份有限公司', '')
    if any(word in w_b_name for word in bank_list) and '银行' in w_b_name:  # 若流水银行名称已经包含分行、支行等内容，则不再搜索明细源文件银行信息
        s_b_name = w_b_name
    else:
        s_b_name1 = ''
        s_b_name2 = ''
        s_b_name3 = ''
        for s_j in range(1, s_ws1.max_column + 1):  # 明细源文件中的银行信息按照 总行、分行、支行进行整合
            cell_ij0 = s_ws1.cell(row=s_i, column=s_j).value
            cell_ij1 = s_ws1.cell(row=s_i, column=s_j+1).value
            cell_ij2 = s_ws1.cell(row=s_i, column=s_j+2).value
            if cell_ij0 is None:  # 空值跳过
                continue
            if '银行' in cell_ij0 and not any(word in cell_ij0 for word in nobank1_list):
                s_b_name1 = cell_ij0.strip()
                # print('1-',s_b_name1)
                if '分行' in cell_ij1 and not any(word in cell_ij1 for word in nobank2_list):
                    s_b_name2 = cell_ij1.strip().replace(s_b_name1, '')
                    # print('2-',s_b_name2)
                if '支行' in cell_ij2 or '营业部' in cell_ij2 or '分行' in cell_ij2:
                    s_b_name3 = cell_ij2.strip().replace(s_b_name1, '').replace(s_b_name2, '')
                    # print('3-',s_b_name3)
                break
        s_b_name123 = (s_b_name1+s_b_name2+s_b_name3)
        # print('4-',s_b_name123)
        s_b_name123 = s_b_name123.replace(w_b_name, '', 3).replace(w_b_name.replace('中国', ''), '', 3).replace('中国', '')  # 去掉银行重复名
        # print('5-',s_b_name123)
        if s_b_name123 != '':
            s_b_name = w_b_name+'('+s_b_name123+')'
        else:
            s_b_name = w_b_name
    s_b_name = s_b_name.replace('有限责任公司', '').replace('股份有限公司', '')
    return s_b_name, s_b_num


def arrange_title(a_file, ap_sub_path):
    a_wb1 = openpyxl.load_workbook(ap_sub_path + a_file)  # 读取明细(流水)源文件
    a_ws1 = a_wb1.active
    ar, ac = 0, 0
    for ai in range(1, a_ws1.max_row):  # 搜寻投资人姓名标题所在列号 ac
        for j in range(1, a_ws1.max_column + 1):
            if a_ws1.cell(row=ai, column=j).value in investor_index_list:
                ac = j
                break
    for ai in range(1, a_ws1.max_row):  # 删除 客户姓名列 所有空白单元格 所在行
        if a_ws1.cell(row=ai, column=ac).value is None or str(a_ws1.cell(row=ai, column=ac).value)[:8] == 'Unnamed:':
            a_ws1.delete_rows(ai)
    a_wb1.save(ap_sub_path + a_file)
    return ac


def to_xlsx(t_file, t_dir_path):  # xls格式转换为xlsx 并去除单元格合并格式
    tx = pd.read_excel(t_dir_path + t_file, header=0, dtype=str, keep_default_na=False)
    t_file = "改————" + t_file
    if t_file.endswith('.xls'):
        t_file = t_file + 'x'
    t_file_path = t_dir_path + t_file
    tx.to_excel(t_file_path, index=False)  # 保存为xlsx文件
    return t_file, t_file_path


def to_pdf(doc_path):  # word文件转化为pdf格式
    word_app = win32com.client.gencache.EnsureDispatch('word.Application')
    word_app.Visible = False
    word_file_path = doc_path
    output_file_path = doc_path[:-5] + '_temp.pdf'  # 定义输入文件和导出文件路径
    id_doc = word_app.Documents.Open(os.path.abspath(word_file_path))  # 通过wordApp打开输入文件
    # 调用myDoc.ExportAsFixedFormat导出文件
    id_doc.ExportAsFixedFormat(os.path.abspath(output_file_path), win32com.client.constants.wdExportFormatPDF,
                               Item=win32com.client.constants.wdExportDocumentWithMarkup,
                               CreateBookmarks=win32com.client.constants.wdExportCreateHeadingBookmarks)
    id_doc.Close()
    word_app.Quit()  # 退出应用
    if '_表章' in doc_path:
        pass
    else:
        pdf_add_steal(output_file_path, doc_path)


def pdf_add_steal(output_pdf_path, id_doc_path):
    output_steal_path = output_pdf_path[:-9] + '_表章_temp.pdf'
    with open(output_pdf_path, 'rb') as pdf_file:  # 打开明细__无章pdf文档
        writer = PyPDF4.PdfFileWriter()  # 创建输出writer
        reader = PyPDF4.PdfFileReader(pdf_file)  # 创建reader
        page_num = reader.numPages  # 获得明细_temp pdf文档页数
        with open(follow_dir_path + 'pdf用章.pdf', 'rb') as seal_file:
            with open(output_steal_path, 'rb') as steal_file:
                seal_reader = PyPDF4.PdfFileReader(seal_file)  # 打开首页，中间页章
                steal_reader = PyPDF4.PdfFileReader(steal_file)  # 打开尾页章
                steal_page_num = steal_reader.numPages  # 获得表章_temp pdf文档页数
                seal_page0 = seal_reader.getPage(0)  # 获取印章文件的第一页 首页章
                seal_page1 = seal_reader.getPage(1)  # 获取印章文件的第二页 中间页章
                steal_page = steal_reader.getPage(page_num-2)  # 获取表章文件的最后一页 尾页章
                for pi in range(0, page_num):  # 按明细_temp pdf文档页数循环
                    page = reader.getPage(pi)
                    if pi == 0:
                        seal_page0.mergePage(page)  # 叠加首页章
                        writer.addPage(seal_page0)  # 将叠加后结果添加到writer
                    elif pi == page_num-1 and steal_page_num < page_num:
                        steal_page.mergePage(page)  # 叠加尾页章
                        writer.addPage(steal_page)  # 将叠加后结果添加到writer
                    else:
                        seal_page2 = copy(seal_page1)  # 复制印章文件的第二页 中间页章
                        seal_page2.mergePage(page)  # 叠加中间页章
                        writer.addPage(seal_page2)  # 将叠加后结果添加到writer
                with open(id_doc_path[:-5] + '.pdf', 'wb') as resultFile:  # 写出到文件
                    writer.write(resultFile)
    print('已生成成立文件：', id_doc_path[:-5] + '.pdf', '\n')
    os.remove(output_pdf_path)
    os.remove(output_steal_path)
    os.remove(output_steal_path[:-9]+'.docx')

    
def word_investor_detail(w_investor_num, wp_full_name, w_p_num, w_id_path, wp_sub_path):
    id_follow_doc_path = follow_dir_path + '投资人明细表（py生成模板）.docx'
    steal_follow_doc_path = follow_dir_path + '投资人明细表（py表章定位模板）.docx'
    id_doc_name = '{}--投资人明细表（第{}次募集）'.format(wp_full_name, num_to_cn(w_p_num))
    steal_doc_name = '{}--投资人明细表（第{}次募集）_表章'.format(wp_full_name, num_to_cn(w_p_num))
    id_doc_path = wp_sub_path + id_doc_name + '.docx'
    steal_doc_path = wp_sub_path + steal_doc_name + '.docx'
    id_doc = docx.Document(id_follow_doc_path)
    steal_doc = docx.Document(steal_follow_doc_path)
    wb = openpyxl.load_workbook(w_id_path)
    ws = wb[wb.sheetnames[len(wb.sheetnames) - 1]]

    id_dict = {}
    for j in range(1, ws.max_column + 1):  # 生成word投资者明细替换内容字典
        id_dict[ws.cell(row=1, column=j).value] = str(ws.cell(row=2, column=j).value).replace('\n', '，').replace('\r', '，')

    for paragraph in id_doc.paragraphs:  # word投资者明细文字内容替换
        for run in paragraph.runs:
            for name in id_dict:
                run.text = run.text.replace('«' + name + '»', id_dict[name])  #

    table = id_doc.tables[0]
    font_name = u'等线'
    for wi in range(2, w_investor_num + 3):  # word投资者明细表格内容替换
        if wi == 3 and ws.cell(row=wi, column=24).value == '合计':
            break
        table.add_row().height = Cm(0.8)
        row_cells = table.rows[wi - 1].cells
        
        for j in range(21 + w2c, 28 + w2c):
            if ws.cell(row=wi, column=j).value is None:
                pass
            elif j == 24 + w2c or j == 25 + w2c:
                run = row_cells[j - 21 - w2c].paragraphs[0].add_run('%.2f' % float(ws.cell(row=wi, column=j).value))
                run.font.name = font_name
                run.font.size = Pt(10)
                row_cells[j - 21 - w2c].paragraphs[0].paragraph_format.space_after = Pt(0)
                row_cells[j - 21 - w2c].paragraphs[0].paragraph_format.alignment = WD_TABLE_ALIGNMENT.CENTER
                row_cells[j - 21 - w2c].vertical_alignment = WD_ALIGN_VERTICAL.CENTER
            else:
                run = row_cells[j - 21 - w2c].paragraphs[0].add_run(str(ws.cell(row=wi, column=j).value))
                run.font.name = font_name
                run.font.size = Pt(10)
                r = run._element
                r.rPr.rFonts.set(qn('w:eastAsia'), font_name)
                row_cells[j - 21 - w2c].paragraphs[0].paragraph_format.space_after = Pt(0)
                row_cells[j - 21 - w2c].paragraphs[0].paragraph_format.alignment = WD_TABLE_ALIGNMENT.CENTER
                row_cells[j - 21 - w2c].vertical_alignment = WD_ALIGN_VERTICAL.CENTER
    
    id_doc.save(id_doc_path)
    
    steal_table = steal_doc.tables[0]
    for wi in range(2, w_investor_num + 3):  # 表格结尾章定位表生成
        if wi == 3 and ws.cell(row=wi, column=24).value == '合计':
            break
        steal_table.add_row().height = Cm(0.8)
        steal_row_cells = steal_table.rows[wi - 1].cells
        
        for j in range(21 + w2c, 27 + w2c):
            if ws.cell(row=wi, column=j).value is None:
                pass
            elif j == 24 + w2c or j == 25 + w2c:
                steal_run = steal_row_cells[j - 21 - w2c].paragraphs[0].add_run('%.2f' % float(ws.cell(row=wi, column=j).value))
                steal_run.font.name = font_name
                steal_run.font.size = Pt(10)
                steal_run.font.color.rgb = RGBColor(255, 255, 255)
                steal_row_cells[j - 21 - w2c].paragraphs[0].paragraph_format.space_after = Pt(0)
                steal_row_cells[j - 21 - w2c].paragraphs[0].paragraph_format.alignment = WD_TABLE_ALIGNMENT.RIGHT
                steal_row_cells[j - 21 - w2c].vertical_alignment = WD_ALIGN_VERTICAL.CENTER
            else:
                steal_run = steal_row_cells[j - 21 - w2c].paragraphs[0].add_run(str(ws.cell(row=wi, column=j).value))
                steal_run.font.name = font_name
                steal_run.font.size = Pt(10)
                steal_run.font.color.rgb = RGBColor(255, 255, 255)
                r = steal_run._element
                r.rPr.rFonts.set(qn('w:eastAsia'), font_name)
                steal_row_cells[j - 21 - w2c].paragraphs[0].paragraph_format.space_after = Pt(0)
                steal_row_cells[j - 21 - w2c].paragraphs[0].paragraph_format.alignment = WD_TABLE_ALIGNMENT.RIGHT
                steal_row_cells[j - 21 - w2c].vertical_alignment = WD_ALIGN_VERTICAL.CENTER
    
    img_run = steal_doc.tables[0].rows[w_investor_num].cells[6].paragraphs[0].add_run()
    img_run.add_picture(follow_dir_path + '司进龙章.png')
    steal_doc_path = steal_doc_path
    steal_doc.save(steal_doc_path)
    
    to_pdf(steal_doc_path)
    to_pdf(id_doc_path)
    

def generate_investor_detail(i_file, ib_w_name, i_p_num, i_p_code, i_p_name, ip_dir_path, ip_sub_path, c):
    investor_detail_name = i_p_code + i_p_name + '募集投资者明细.xlsx'
    investor_detail_path = ip_dir_path + investor_detail_name  # 各次投资者明细汇总文件路径（不含公式）
    generate_name = 'xxx-{}{}次募集投资者明细.xlsx'.format(i_p_name, i_p_num)
    generate_path = ip_dir_path + generate_name  # 某次产品投资者明细excel生成文件路径
    shutil.copy(follow_dir_path + 'xxx-XX募集投资者明细+要素表.xlsx', generate_path)  # 复制明细模板至产品文件夹
    wb = openpyxl.load_workbook(ip_sub_path + ib_w_name)  # 读取流水合并文件‘改——’
    ws = wb.active
    wb1 = openpyxl.load_workbook(ip_sub_path + i_file)  # 读取明细文件'改——'
    ws1 = wb1.active
    wb2 = openpyxl.load_workbook(generate_path)  # 读取明细生成模板
    ws2 = wb2['生成模板']

    investor_list = []
    amount_list = []
    bank_list = []
    bank_num_list = []
    ID_type_list = []
    ID_num_list = []

    lack_bank = 0
    lack_ID = 0
    investor_num = 0
    for gi in range(2, ws1.max_row + 1):  # 投资人姓名
        if ws1.cell(row=gi, column=c).value is None or ws1.cell(row=gi, column=c).value == 'nan':
            break
        else:
            ws2.cell(row=gi, column=22 + w2c).value = ws1.cell(row=gi, column=c).value.strip().replace(' ', '')
            investor_num += 1
            investor_list.append(ws2.cell(row=gi, column=22 + w2c).value)

    for j in range(1, ws1.max_column + 1):  # 证件类型
        if ws1.cell(row=1, column=j).value in ID_type_index_list:
            for gi in range(2, investor_num + 2):
                ws2.cell(row=gi, column=13).value = ws1.cell(row=gi, column=j).value
                ID_type_list.append(ws2.cell(row=gi, column=13).value)
            break

    lack_all_ID = 1
    for j in range(1, ws1.max_column + 1):  # 证件号码
        if ws1.cell(row=1, column=j).value in ID_num_index_list:
            lack_all_ID = 0
            for gi in range(2, investor_num + 2):
                ws2.cell(row=gi, column=14).value = ws1.cell(row=gi, column=j).value
                ID_num_list.append(ws2.cell(row=gi, column=14).value)
                if ws2.cell(row=gi, column=14).value == '空':
                    lack_ID += 1
            break
    if lack_all_ID == 1:
        lack_ID = lack_ID + investor_num
        ID_type_list = ['空'] * investor_num
        ID_num_list = ['空'] * investor_num

    t = 0
    for j in range(1, ws1.max_column + 1):  # 认购本金
        if ws1.cell(row=1, column=j).value in amount_index_list and ws1.cell(row=2, column=j).value is not None:
            for gi in range(2, investor_num + 2):
                ij = int(float(ws1.cell(row=gi, column=j).value))
                if ij >= 30000:
                    ws2.cell(row=gi, column=23 + w2c).value = ij / 10000
                else:
                    ws2.cell(row=gi, column=23 + w2c).value = ij
                if ws2.cell(row=gi, column=23 + w2c).value is None:
                    ID_type_list.append('nan')
                else:
                    amount_list.append(ws2.cell(row=gi, column=23 + w2c).value)
                t = t + ws2.cell(row=gi, column=23 + w2c).value
            break
    clean_rows(ws2, investor_num + 2, ws2.max_row)
    ws2.cell(row=investor_num + 2, column=21 + w2c).value = '合计'
    ws2.cell(row=investor_num + 2, column=23 + w2c).value = t  # 认购本金合计
    ws2.cell(row=investor_num + 2, column=24 + w2c).value = '=SUM({0}2:{0}{1})'.format(
        openpyxl.utils.get_column_letter(24 + w2c), investor_num + 1)  # 预期收益合计
    ws2.cell(row=investor_num + 2, column=25 + w2c).value = '=SUM({0}2:{0}{1})'.format(
        openpyxl.utils.get_column_letter(25 + w2c), investor_num + 1)  # 本金及收益合计

    e, f, g, h = 0, 0, 0, 0
    for j in range(1, ws.max_column + 1):
        if ws.cell(row=1, column=j).value == '付款方别（户）名':  # 流水表第7列
            e = j
        elif ws.cell(row=1, column=j).value == '付款方账号':  # 流水表第8列
            f = j
        elif ws.cell(row=1, column=j).value == '付款方银行':  # 流水表第9列
            g = j
        elif ws.cell(row=1, column=j).value == '交易金额':  # 流水表第10列
            h = j

    for gi in range(2, investor_num + 2):  # 回款账户开户行、回款账户号码
        m = 0  # 该投资人最大单笔流水金额记录器
        bt = 0  # 该投资人银行流水金额合计
        for b in range(2, ws.max_row + 1):
            if ws.cell(row=b, column=e).value == ws2.cell(row=gi, column=22 + w2c).value:
                bh = int(float(ws.cell(row=b, column=h).value))  # 该投资人单笔流水交易金额
                bt = bt + ws.cell(row=b, column=h).value
                s_bank_name, s_bank_num = search_bank(ws1, gi, ws.cell(row=b, column=g).value)  # 搜索明细源文件中银行信息
                if m < bh:  # 获得交易金额更大的单笔流水所对应的银行卡号
                    if ws.cell(row=b, column=f).value is not None:  # 流水付款方账号不为空
                        ws2.cell(row=gi, column=26 + w2c).value = ws.cell(row=b, column=f).value
                    else:
                        ws2.cell(row=gi, column=26 + w2c).value = '流水无卡号' + '/' + s_bank_num
                    ws2.cell(row=gi, column=27 + w2c).value = s_bank_name
                    m = bh

        bank_num_list.append(ws2.cell(row=gi, column=26 + w2c).value)
        bank_list.append(ws2.cell(row=gi, column=27 + w2c).value)
        if ws2.cell(row=gi, column=8 + w2c).value == '空':
            lack_bank += 1
        elif ws2.cell(row=gi, column=9+w2c).value == '空':
            lack_bank += 1
        if bt / 10000 != ws2.cell(row=gi, column=23 + w2c).value:
            print('{}银行流水合计{}（万元）与认购本金{}（万元）不相等，请核对。'.format(ws2.cell(row=gi, column=22 + w2c).value,
                                                             bt / 10000, ws2.cell(row=gi, column=23 + w2c).value))

    data_dict = {'姓名': investor_list, '投资额(万元)': amount_list, '银行账号': bank_num_list, '银行名称': bank_list,
                 '证件类型': ID_type_list, '证件编号': ID_num_list}
    all_data = pd.DataFrame(data_dict)
    all_data.index = all_data.index + 1
    display(all_data)
    # print(all_data)

    ws2.cell(row=3, column=33 + w2c).value = i_p_name  # 输入产品名简称
    ws2.cell(row=3, column=35 + w2c).value = i_p_num  # 输入产品期数
    ws2.cell(row=3, column=37 + w2c).value = issuer_name  # 输入发行人简称
    ws2.cell(row=2, column=34 + w2c).value = t  # 输入金额合计

    wb2.save(generate_path)
    refresh(generate_path)
    wb2 = openpyxl.load_workbook(generate_path, data_only=True)
    ws2 = wb2['生成模板']
    wb2.remove(wb2['要素表'])

    # 投资者明细excel去公式
    if investor_detail_name not in os.listdir(ip_dir_path):  # 如果文件夹中没有投资者明细excel文件，就复制一个模版过来
        shutil.copy(follow_dir_path + '首次募集投资者明细.xlsx', investor_detail_path)
    wb3 = openpyxl.load_workbook(investor_detail_path)
    for sheetname in wb3.sheetnames:
        if '第{}次'.format(num_to_cn(i_p_num)) in sheetname:  # 如果投资者明细excel文件已有该产品工作表，直接删除
            wb3.remove(wb3[sheetname])
    ws3 = wb3.copy_worksheet(wb3.worksheets[0])  # 复制工作簿内模版工作表，并命名
    ws3.title = '第{}次{}'.format(num_to_cn(i_p_num), ws2.cell(row=2, column=34 + w2c).value)

    for gi, row in enumerate(ws2.iter_rows()):  # 复制工作表
        for j, cell in enumerate(row):
            ws3.cell(row=gi + 1, column=j + 1).value = cell.value
    wb3.save(investor_detail_path)

    border = Border(left=Side(border_style='thin', color='000000'),
                    right=Side(border_style='thin', color='000000'),
                    top=Side(border_style='thin', color='000000'),
                    bottom=Side(border_style='thin', color='000000'))  # 加边框
    font4 = Font(size="9")
    align = Alignment(horizontal='center', vertical='center', wrap_text=False)  # 居中 不自动换行

    # 更新统计表
    if '统计表.xlsx' not in os.listdir(found_dir_path):  # 将投资者明细excel文件第28-42列内容复制到统计表中
        shutil.copy(follow_dir_path + '统计表.xlsx', found_dir_path)
    wb4 = openpyxl.load_workbook(found_dir_path + '统计表.xlsx')
    ws4 = wb4.active
    pdf_save_path = ip_sub_path + ws2.cell(row=2, column=41 + w2c).value + '.pdf'  # pdf文件路径保存
    
    file_name_list = []
    for gi in range(2, ws4.max_row + 1):  # 判断统计表中是否含有该产品信息
        file_name_list.append(ws4.cell(row=gi, column=14).value)
    if pdf_save_path not in file_name_list:  # 若无该产品信息，行号追加模式
        r4 = ws4.max_row + 1
    else:
        r4 = file_name_list.index(pdf_save_path) + 2  # 则行号取该产品信息行号，覆盖模式
        
    for j in range(28 + w2c, 46 + w2c):  # 循环跨表复制信息
        ws4.cell(row=r4, column=j-27-w2c).value = ws2.cell(row=2, column=j).value
        ws4.cell(row=r4, column=j-27-w2c).font = font4
        ws4.cell(row=r4, column=j-27-w2c).alignment = align
        ws4.cell(row=r4, column=j-27-w2c).border = border
        
    ws4.cell(row=r4, column=14).value = pdf_save_path
    ws4.cell(row=r4, column=14).font = font4
    ws4.cell(row=r4, column=14).alignment = align
    ws4.cell(row=r4, column=14).border = border
    
    ws4.cell(row=r4,column=19).value = \
        '=ROUND(IF(DATEDIF(C{0},K{0},"m")>12,$AD$1*G{0},IF(RIGHT(I{0},1)="月",$AD$1*G{0}*DAT'\
        'EDIF(C{0},K{0},"m")/12,$AD$1*G{0}*DATEDIF(C{0},K{0},"d")/365))*10000,2)'.format(r4)
    ws4.cell(row=r4, column=19).font = font4  # 管理费计算公式
    ws4.cell(row=r4, column=19).alignment = align
    ws4.cell(row=r4, column=19).border = border
    
    ws4.cell(row=r4, column=20).value = lack_ID  # 缺身份证信息人数
    ws4.cell(row=r4, column=20).font = font4
    ws4.cell(row=r4, column=20).border = border
    if lack_ID == 0:
        ws4.cell(row=r4, column=20).alignment = Alignment(horizontal='right', vertical='center', wrap_text=False)
    else:
        ws4.cell(row=r4, column=20).alignment = Alignment(horizontal='center', vertical='center', wrap_text=False)
    
    ws4.cell(row=r4, column=21).value = lack_bank  # 缺银行账户信息人数
    ws4.cell(row=r4, column=21).font = font4
    ws4.cell(row=r4, column=21).border = border
    if lack_bank == 0:
        ws4.cell(row=r4, column=21).alignment = Alignment(horizontal='right', vertical='center', wrap_text=False)
    else:
        ws4.cell(row=r4, column=21).alignment = Alignment(horizontal='center', vertical='center', wrap_text=False)
    
    ws4.cell(row=r4, column=22).value = '无'  # 有无银行回单
    ws4.cell(row=r4, column=22).font = font4
    ws4.cell(row=r4, column=22).border = border
    ws4.cell(row=r4, column=22).alignment = Alignment(horizontal='left', vertical='center', wrap_text=False)
    
    wb4.save(found_dir_path + '统计表.xlsx')

    # 更新核心数据
    if '核心数据.xlsx' not in os.listdir('../'):  # 判断是否有核心数据表，若没有则复制模版
        shutil.copy(follow_dir_path + '核心数据.xlsx', '../')
    wb5 = openpyxl.load_workbook('../核心数据.xlsx')
    ws5 = wb5.active
    investor_code_list = []
    for gi in range(2, ws5.max_row + 1):  # 核心数据现有投资人名称列表
        investor_code_list.append(ws5.cell(row=gi, column=2).value)
    
    # 判断是追加写入还是覆盖插入信息
    for gi in range(2, investor_num + 2):
        i_code = ws2.cell(row=gi, column=2).value  # 明细表中投资人统编号
        i_ID = ws2.cell(row=gi, column=13).value  # 明细表中投资人身份证号
        if i_code not in investor_code_list:  # 核心数据中无该投资人统编号
            r5 = ws5.max_row + 1
        else:
            if investor_code_list.count(i_code) > 1:  # 该产品中有同名投资人
                if i_ID == ws5.cell(row=investor_code_list.index(i_code) + 2, column=13).value:  # 有身份证号且相同
                    r5 = investor_code_list.index(i_code) + 2
                else:  # 无身份证信息
                    print('发现同名投资人{}，且身份信息无法分辨，请确认并手动区分'.format(i_code))
                    sys.exit(0)
            else:  # 该产品中无同名投资人
                r5 = investor_code_list.index(i_code) + 2
                
        for j in range(1, 21+w2c):  # 复制明细表该行信息至核心数据表
            if j == 6:  # 起息日
                ws5.cell(row=r5, column=6).value = ws2.cell(row=gi, column=6).value.date()
            elif j == 7:  # 到期日
                ws5.cell(row=r5, column=7).value = ws2.cell(row=gi, column=7).value.date()
            elif j == 8:  # 利率
                ws5.cell(row=r5, column=8).value = ws2.cell(row=gi, column=8).value
                ws5.cell(row=r5, column=8).number_format = '0.00%'  # 利率列显示为2位小数百分比                      
            else:
                ws5.cell(row=r5, column=j).value = ws2.cell(row=gi, column=j).value
        for j in range(1, 26+w2c):  # 核心数据表格式设置 加边框和文字对齐
            ws5.cell(row=r5, column=j).alignment = align
            ws5.cell(row=r5, column=j).border = border

    wb5.save('../核心数据.xlsx')
    os.remove(generate_path)  # 删除投资者明细源excel中间文件‘改——’
    os.remove(ip_sub_path + ib_w_name)  # 删除银行流水excel中间文件‘改——’
    os.remove(ip_sub_path + i_file)  # 删除投资者明细合成excel带公式文件 ‘xxx-’
    word_investor_detail(investor_num, product_full_name, product_num, investor_detail_path,
                         product_sub_path)  # 生成word投资者明细


def draw_chart():
    plt.rcParams['font.family']=['FangSong']  # 显示汉字字体
    plt.rcParams['font.size']=15  # 字号
    found_df = pd.read_excel(found_dir_path + '统计表.xlsx', usecols='A:V')
    cn_date = '{}年{}月{}日'.format(date.today().year, date.today().month, date.today().day)
    today_found_obj = found_df.groupby('起息日').get_group(cn_date).copy()
    # 绘制柱状图
    today_found_obj['发行人'] = today_found_obj.产品名称.map(lambda x: x[:-2])
    amount_sum = today_found_obj['实际募集金额'].sum()
    found_amount = today_found_obj.sort_values(by='发行人').groupby('发行人')['实际募集金额'].sum()
    investor_num = today_found_obj.sort_values(by='发行人').groupby('发行人')['投资者数量'].sum()
    plt.figure(figsize=(10, 5))
    plt.bar(found_amount.index, found_amount)
    plt.title('{}各发行人产品成立规模统计,合计：{}(单位：万元）'.format(cn_date,amount_sum))
    plt.ylabel('成立规模(万元)')
    plt.xlabel('发行人')
    for i in found_amount.index:
        plt.text(i, found_amount[i], '{}({}人)'.format(found_amount[i],investor_num[i]), ha='center', va='bottom')
    plt.show()
    # 绘制饼图
    wealth_amount = today_found_obj.groupby('财富机构')['实际募集金额'].sum().sort_values()
    wealth_amount
    plt.figure(figsize=(10, 8))
    plt.pie(wealth_amount, labels = wealth_amount.index, autopct='%.2f%%')
    plt.title('{}各财富机构产品成立规模统计,合计：{}(单位：万元）'.format(cn_date,amount_sum))
    plt.axis('equal')
    plt.show()
              

d1 = datetime.strptime('2021-10-30', '%Y-%m-%d')
if (datetime.today() - d1).days > 0:
    print('授权已过期！')
    sys.exit(0)
else:
    print('成立机器人启动！')        
        
for file, file_path0 in source_all_file():  # 找出源文件夹中问题文件，并警示
    if product_name_judge(file) == '无法识别':
        print('源文件夹中包含无产品简称文件**{}**，请确认并手动更名或删除后再运行本程序！'.format(file))
        sys.exit(0)
    if file.count('次') + file.count('期') > 1:
        print('源文件夹中包含多期数单位（次、期）文件**{}**，请确认并手动更名或删除后再运行本程序！'.format(file))
        sys.exit(0)
print('源文件检查完毕！')

end = time.time()
print("Running time: %.2fs seconds"%(end - start))

成立机器人启动！
源文件检查完毕！
Running time: 0.35s seconds


# 成立程序

In [2]:
# 成立程序
start = time.time()
i = 0
for file, file_path0 in source_all_file():  # 源文件夹文件包含产品名简称
    product_num = 0
    issuer_name = ''
    if any(word in file for word in period_unit_list):  # 该文件包含“次”或“期”或‘开’字（期数单位）
        print(file)
        
        # 识别发行人
        product_name, p_name_judge, issuer_name2 = product_name_judge(file)  # 提取该文件名中的产品简称, 判断简称是否唯一,通过期数推理发行人简称
        if p_name_judge == '不唯一':
            issuer_repeat_list = df.groupby('产品简称').get_group(product_name)['发行人简称'].tolist()
            issuer_repeat = '/'.join(issuer_repeat_list)
            print('产品简称“{}”对应发行人不唯一，自动尝试从文件名中提取发行人简称。'.format(product_name))
            issuer_name1 = issuer_judge(file)  # 提取该文件名中的发行人简称
            if issuer_name1 == '未能识别':
                issuer_name = issuer_name2
            else:
                issuer_name = issuer_name1
            print('发行人识别结果：文件名识别发行人为“{}”；期数推断发行人为“{}”'.format(issuer_name1, issuer_name2))
            
            # 输入发行人
            if issuer_name == '未能识别':
                issuer_input = input('请输入发行人简称（若不是成立明细信息，可回车跳过，例：{}）：'.format(issuer_repeat))  # 手动输入该产品发行人简称
                while True:
                    try:
                        if issuer_input == '':
                            break
                        elif issuer_input in issuer_repeat_list:
                            issuer_name = issuer_input
                            print('发行人输入为：', issuer_name)
                            print('请检查{}产品期数是否连续！'.format(issuer_name + product_name))
                            break
                    except ValueError:
                        print('发行人简称输入范围：', issuer_repeat)
                        issuer_input = input('输入错误，请按列表范围输入发行人简称（或回车跳过）：')
        else:
            issuer_name = df['产品简称'][df['产品简称'].values == product_name].index[0][:-2]
        
        # 识别或输入期数
        if issuer_name != '':
            try:
                product_num = product_num_judge(file)
                i += 1
                print('******************第{}个产品******************'.format(i))
                print('产品期数识别为：第{}期'.format(product_num))
            except (ValueError, KeyError, IndexError):
                num_input = input('请输入期数（若不是成立明细信息，可回车跳过）：')  # 手动输入该产品数字期数
                while True:
                    try:
                        if num_input == '':
                            break
                        else:
                            product_num = int(num_input)
                            i += 1
                            print('******************第{}个产品******************'.format(i))
                            break
                    except ValueError:
                        num_input = input('输入错误，请输入阿拉伯数字的期数（或回车跳过）：')

    # 无期数单位 但疑似成立信息文件   
    elif len(re.findall(r"[\u4e00-\u9fa5]", file)) > 3 and '流水' not in file:  # 疑似明细源文件（文件名汉字数大于3且无‘流水’）
        print(file)
        
        # 识别发行人
        product_name, p_name_judge, issuer_name2 = product_name_judge(file)  # 提取该文件名中的产品简称, 判断简称是否唯一
        if p_name_judge == '不唯一':
            issuer_repeat_list = df.groupby('产品简称').get_group(product_name)['发行人简称'].tolist()
            issuer_repeat = '/'.join(issuer_repeat_list)
            print('产品简称“{}”对应发行人不唯一，自动尝试从文件名中提取发行人简称。'.format(product_name))
            issuer_name1 = issuer_judge(file)  # 提取该文件名中的发行人简称
            if issuer_name1 == '未能识别':
                issuer_name = issuer_name2
            else:
                issuer_name = issuer_name1
            print('发行人识别结果：文件名识别发行人为“{}”；期数推断发行人为“{}”'.format(issuer_name1, issuer_name2))
            
            # 输入发行人
            if issuer_name == '未能识别':
                issuer_input = input('请输入发行人简称（若不是成立明细信息，可回车跳过，例：{}）：'.format(issuer_repeat))  # 手动输入该产品发行人简称
                while True:
                    try:
                        if issuer_input == '':
                            break
                        elif issuer_input in issuer_repeat_list:
                            issuer_name = issuer_input
                            print('发行人输入为：', issuer_name)
                            print('请检查{}产品期数是否连续！'.format(issuer_name + product_name))
                            break
                    except ValueError:
                        print('发行人简称输入范围：', issuer_repeat)
                        issuer_input = input('输入错误，请按列表范围输入发行人简称（或回车跳过）：')
        else:
            issuer_name = df['产品简称'][df['产品简称'].values == product_name].index[0][:-2]
        
        # 识别或输入期数
        if issuer_name != '':  
            try:
                product_num = product_num_judge(file)
                i += 1
                print('******************第{}个产品******************'.format(i))
                print('产品期数识别为：第{}期'.format(product_num))
            except (ValueError, KeyError, IndexError):
                num_input = input('请输入期数（若不是成立明细信息，可回车跳过）：')  # 手动输入该产品数字期数
                while True:
                    try:
                        if num_input == '':
                            break
                        else:
                            product_num = int(num_input)
                            i += 1
                            print('******************第{}个产品******************'.format(i))
                            break
                    except ValueError:
                        num_input = input('输入错误，请输入阿拉伯数字的期数（或回车跳过）：')
    
    if product_num != 0:
        product_full_name = issuer_name + product_name  # 合成产品名称（发行人简称+产品简称）
        product_code = df.loc[product_full_name, '产品统编号'][:6]  # 提取产品简称对应的产品编号
        product_wealth_management = df.loc[product_full_name, '财富公司']  # 提取产品简称对应的财富公司
        product_dir = product_code + product_full_name + product_wealth_management  # 生成产品文件夹名
        product_dir_path = found_dir_path + product_dir + '/'  # 生成产品文件夹路径
        if product_dir not in os.listdir(found_dir_path):  # 遍历成立文件夹，若产品文件夹不存在
            os.makedirs(product_dir_path)  # 创建产品文件夹
        product_sub_dir = '{}第{}次募集{}'.format(product_name, num_to_cn(product_num), found_date)  # 生成产品子文件夹名
        product_sub_path = product_dir_path + product_sub_dir + '/'  # 生成产品子文件夹路径
        print(product_sub_dir)
        
        if product_sub_dir not in os.listdir(product_dir_path):  # 若产品文件夹中无产品子文件夹
            os.makedirs(product_sub_path)  # 创建产品子文件夹
        else:
            delete_dir = input('即将覆盖同期数产品文件夹内容（回车确认执行，输入其它字符后回车则中止）：')
            if delete_dir == '':
                for f in os.listdir(product_sub_path):
                    if f.endswith('.xls'):
                        shutil.copy(product_sub_path + f, source_dir_path)
            else:
                print('程序中止！建议核对成立信息文件中产品简称及期数！')
                break    
                
        shutil.copy(file_path0, product_sub_path)  # 明细源文件移至产品子文件夹
        file, file_path = to_xlsx(file, product_sub_path)  # 明细源文件去除合并格式变为xlsx
        mc = arrange_title(file, product_sub_path)
        bank_water_name = bank_water_merge(file, product_name, product_num, product_sub_path, mc)
        if bank_water_name == '无流水':
            print('无该产品流水，请添加流水文件至源文件夹！')
            sys.exit(0)
        else:
            generate_investor_detail(file, bank_water_name, product_num, product_code, product_name,
                                     product_dir_path, product_sub_path, mc)
            os.remove(file_path0)
print('成立程序完成！')
draw_chart()
end = time.time()
print("Running time: %.2fs seconds"%(end - start))

西游记爱3——第十八期.xlsx
产品简称“爱3”对应发行人不唯一，自动尝试从文件名中提取发行人简称。
发行人识别结果：文件名识别发行人为“西游记”；期数推断发行人为“未能识别”
******************第1个产品******************
产品期数识别为：第18期
爱3第十八次募集2021-09-23


姓名  投资额(万元)                银行账号        银行名称 证件类型     证件编号
1  黑熊怪      100  622260000000000012  西游记银行花果山支行  身份证  1000001
2  红孩儿      200  622260000000000013  西游记银行高老庄支行  身份证  1000002
3  牛魔王      100  622260000000000014  西游记银行流沙河支行  身份证  1000003
4  小甜甜      300  622260000000000015   西游记银行唐朝支行  身份证  1000004
5   菩提      200  622260000000000016  西游记银行宛子山支行  身份证  1000005

已生成成立文件： ../成立/001003西游记爱3坦克/爱3第十八次募集2021-09-23/西游记爱3--投资人明细表（第十八次募集）.pdf 

西游记爱3——第十六期.xlsx
产品简称“爱3”对应发行人不唯一，自动尝试从文件名中提取发行人简称。
发行人识别结果：文件名识别发行人为“西游记”；期数推断发行人为“未能识别”
******************第2个产品******************
产品期数识别为：第16期
爱3第十六次募集2021-09-23


姓名  投资额(万元)                银行账号        银行名称 证件类型     证件编号
1   孙悟空      100  622260000000000001  西游记银行花果山支行  身份证  1000001
2   猪八戒      200  622260000000000002  西游记银行高老庄支行  身份证  1000002
3    沙僧      170  622260000000000003  西游记银行流沙河支行  身份证  1000003
4    唐僧      100  622260000000000004   西游记银行唐朝支行  身份证  1000004
5   白骨精      500  622260000000000005  西游记银行宛子山支行  身份证  1000005
6    观音      500  622260000000000006  西游记银行花果山支行  身份证  1000006
7    李靖      100  622260000000000007  西游记银行高老庄支行  身份证  1000007
8    哪吒      100  622260000000000008  西游记银行流沙河支行  身份证  1000008
9   哮天犬      100  622260000000000009   西游记银行唐朝支行  身份证  1000009
10   如来      100  622260000000000010  西游记银行宛子山支行  身份证  1000010

已生成成立文件： ../成立/001003西游记爱3坦克/爱3第十六次募集2021-09-23/西游记爱3--投资人明细表（第十六次募集）.pdf 

成立程序完成！


Running time: 40.76s seconds


# 银行回单归档

In [2]:
# 回单归档程序
import requests
import openpyxl
import pandas as pd
import base64
import shutil
import time
import os
from datetime import date
from urllib.parse import quote

API_KEY='BtVbDiuNUMzxvRGsfIH4qKZ7'
SECRET_KEY='BauTCGrfGk3NnTTGsYokhBkGNFx8VIne'
r_align = openpyxl.styles.Alignment(horizontal='right', vertical='center', wrap_text=False)  # 居右 不自动换行

# 获取access_token
def get_access_token():
    # client_id 为官网获取的AK， client_secret 为官网获取的SK
    host = 'https://aip.baidubce.com/oauth/2.0/token?grant_type=client_credentials&client_id={}&client_secret={}'.format(API_KEY, SECRET_KEY)
    response = requests.get(host)
    if response:
        access_token = response.json()['access_token']
        return access_token


def  bank_advice_info(bank_advice_path):
    # POST目标地址
    request_url = 'https://aip.baidubce.com/rest/2.0/solution/v1/iocr/recognise'
    # 二进制方式打开回单图片文件
    f = open(bank_advice_path, 'rb')
    # 对文件进行base64编码
    img = base64.b64encode(f.read())
    # 识别模板编号
    templateSign = "24c5832a27d640aa6aec8be05acfed1e"

    # 创建请求参数并通过POST方法发送
    access_token = get_access_token()
    params = "access_token=" + access_token + "&templateSign=" + templateSign + "&image=" + quote(img) # 其他写法{'image=': img}
    headers = {'Content-Type': "application/x-www-form-urlencoded",'charset': "utf-8"}
    response = requests.post(request_url, data=params, headers=headers)
    f.close()
    
    product_name = '空'
    # 获取结果
    while True:
        if response:
            ret = response.json()['data']['ret']
            amount = ret[0]['word'][:-2].replace(',', '').replace('.', '')  # 回单金额整数部分
            for word in product_list:
                if word in ret[1]['word']:
                    product_name = word  # 回单产品名称
            y, m, d = time.strptime(ret[2]['word'], '%Y%m%d')[0:3]  # 回单日期
            cn_date = '{}年{}月{}日'.format(y, m, d)
            return cn_date, amount, product_name
            break
        else:
            time.sleep(1)

            
source_dir_path = './源文件/'
follow_dir_path = './模板/'  # 模板文件夹
product_element_path = follow_dir_path + 'xxx-XX募集投资者明细+要素表.xlsx'  # 要素表
df = pd.read_excel(product_element_path, sheet_name='要素表',dtype={2: str}).set_index('产品简称')  # 读取要素表并以‘产品简称’列为标签列
product_list = df.index.dropna().values.tolist()  # 从要素表提取产品名简称列表

df_a = pd.read_excel('../成立/统计表.xlsx', usecols='A:V', dtype=str)
wb_a = openpyxl.load_workbook('../成立/统计表.xlsx')
ws_a = wb_a['统计表']
token = get_access_token()

for file in os.listdir(source_dir_path):
    if file.endswith('.png'):
        advice_path = source_dir_path + file
        a_date, a_amount, a_product_name = bank_advice_info(advice_path)
        print('===获取{}的银行回单==='.format(a_product_name))
        print('回单金额：{}元'.format(a_amount))
        print('回单日期：', a_date)
        try:
            a_group_df = df_a.groupby('起息日').get_group(a_date)
        except KeyError:
            print('统计表中未发现{}成立产品'.format(a_date))
            continue
        for index, row in a_group_df.iterrows():
            a_text_dict = dict(row)
            if a_product_name in a_text_dict['产品名称'] and int(a_amount) == float(a_text_dict['实际募集金额'])*10000:
                a_move_path = row[13][:-4].replace('--投资人明细表', '') + '银行回单.png'  # 回单文件的路径
                ws_a.cell(row=index+2, column=22).value =  '有'  # 统计表标注有回单
                ws_a.cell(row=index+2, column=22).alignment = r_align
                shutil.move(advice_path, a_move_path)
                print('回单保存至：', a_move_path)
wb_a.save('../成立/统计表.xlsx')
print('银行回单归档完成！')

KeyboardInterrupt: 

# 清理缓存

In [7]:
# 清理缓存文件夹
from win32com.client.gencache import EnsureDispatch
import sys
import os
import shutil
xl = EnsureDispatch("Word.Application")
print('删除缓冲文件夹：', os.path.dirname(sys.modules[xl.__module__].__file__))
shutil.rmtree(r'C:\Users\Administrator\AppData\Local\Temp\gen_py\3.6')

删除缓冲文件夹： C:\Users\Administrator\AppData\Local\Temp\gen_py\3.6\00020905-0000-4B30-A977-D214852036FFx0x3x0
